# Preprocesar los datos

In [1]:
import xarray as xr
import pandas as pd

### Fechas

In [3]:
def preprocess_dates(boya, copernicus, gow):
    # Establecer el rango de fechas común
    time_min = max(boya.time.min(), copernicus.time.min(), gow.time.min())
    time_max = min(boya.time.max(), copernicus.time.max(), gow.time.max())
    
    boya = boya.sel(time=slice(time_min, time_max))
    copernicus = copernicus.sel(time=slice(time_min, time_max))
    gow = gow.sel(time=slice(time_min, time_max))
    
    # Como los datos tiene huecos, los quitamos
    boya_not_null = boya['hs'].notnull()
    boya = boya.where(boya_not_null, drop=True)
    copernicus = copernicus.where(boya_not_null, drop=True)
    gow = gow.where(boya_not_null, drop=True)
    
    boya_not_null = boya['dir'].notnull()
    boya = boya.where(boya_not_null, drop=True)
    copernicus = copernicus.where(boya_not_null, drop=True)
    gow = gow.where(boya_not_null, drop=True)
    
    copernicus_not_null = copernicus['VHM0'].notnull()
    boya = boya.where(copernicus_not_null, drop=True)
    copernicus = copernicus.where(copernicus_not_null, drop=True)
    gow = gow.where(copernicus_not_null, drop=True)
    
    gow_not_null = gow['hs'].notnull()
    boya = boya.where(gow_not_null, drop=True)
    copernicus = copernicus.where(gow_not_null, drop=True)
    gow = gow.where(gow_not_null, drop=True)
    
    return boya, copernicus, gow

In [4]:
df_boyas = pd.read_csv('boyas.csv')
for nombre in df_boyas.Nombre:
    print(nombre)
    boya = xr.open_dataset(f'data/raw/boyas/{nombre}_Ext.nc')
    copernicus = xr.open_dataset(f'data/raw/historico/IBI_REANALYSIS_WAV_005_006_{nombre}.nc')
    gow = xr.open_dataset(f'data/raw/historico/GOW_CFS_{nombre}.nc')

    boya, copernicus, gow = preprocess_dates(boya, copernicus, gow)
    (len(boya.time), len(copernicus.time), len(gow.time))
    
    # Seleccionar varibables de interes
    boya = boya[['hs', 't02', 'tp', 'dir']]
    copernicus = copernicus[['VHM0', 'VTM02', 'VTPK', 'VMDR']]
    
    # Pasar fp a tp y Seleccionar varibables de interes
    gow['tp'] = 1 / gow['fp']
    gow = gow[['hs', 't02', 'tp', 'dir']]
       
    boya.to_netcdf(f'data/processed/boya_{nombre}_Ext.nc', )
    copernicus.to_netcdf(f'data/processed/IBI_REANALYSIS_WAV_005_006_{nombre}.nc')
    gow.to_netcdf(f'data/processed/GOW_CFS_{nombre}.nc')

Bilbao-Vizcaya
Cabo_de_Pennas
Estaca_de_Bares
Villano-Sisargas
Cabo_Silleiro
Golfo_de_Cadiz
Cabo_de_Gata
Cabo_de_Palos
Valencia
Tarragona
Cabo_Begur
Dragonera
Mahon
Gran_Canaria
Tenerife
